# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
part_one = os.path.join("..", "output_data","cities.csv")
weather_df = pd.read_csv(part_one, delimiter=",")
weather_df.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Puerto Ayora,-0.7393,-90.3518,73.99,92,25,3.00,EC,1610251685
1,1,Upernavik,72.7868,-56.1549,10.47,83,96,8.99,GL,1610251854
2,2,Leiyang,26.4024,112.8591,43.83,55,99,5.53,CN,1610252034
3,3,Ushuaia,-54.8000,-68.3000,44.60,87,20,16.11,AR,1610251823
4,4,Bluff,-46.6000,168.3333,64.00,70,93,23.00,NZ,1610251874


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)

In [11]:

coordinates = weather_df[["Latitude","Longitude"]].astype(float)
humidity = weather_df ["Humidity"].astype(float)
max_humidity=humidity.max()


fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating = False, max_intensity = max_humidity, point_radius = 1)

fig.add_layer(heat_layer)
fig
#############?????????the key is correct.

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_weather_df = weather_df.loc[(weather_df["Max Temperature"] > 70) 
                                   & (weather_df["Max Temperature"] < 80)
                                  & (weather_df["Cloudiness"] ==0),:]

                                   
narrow_weather_df = narrow_weather_df.dropna()
narrow_weather_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,Hobyo,5.3505,48.5268,75.06,72,0,22.41,SO,1610252173
86,86,Kidal,18.4411,1.4078,72.05,20,0,10.54,ML,1610252271
100,100,Cidreira,-30.1811,-50.2056,77.04,80,0,18.37,BR,1610252290
133,133,Araouane,18.9048,-3.5265,70.18,21,0,11.54,ML,1610252332
227,227,Richards Bay,-28.7830,32.0377,71.24,83,0,6.71,ZA,1610252456
241,241,Bassila,9.0081,1.6654,71.60,68,0,2.73,BJ,1610252474
333,333,El Salvador,13.8333,-88.9167,78.80,57,0,8.05,SV,1610252509
381,381,Formosa,-26.1775,-58.1781,75.20,100,0,3.44,AR,1610252658
393,393,Barão de Melgaço,-16.1944,-55.9675,78.80,89,0,3.44,BR,1610252674
438,438,Sambava,-14.2667,50.1667,78.08,77,0,4.03,MG,1610252732


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrow_weather_df
##hotel_df



In [7]:
hotel_list = []

for index, row in hotel_df.iterrows():
    hotel_lat = row["Latitude"]
    hotel_lng = row["Longitude"]
    
    params = {
        "location" : f"{hotel_lat},{hotel_lng}",
        "radius" : 5000,
        "type" : "hotel",
        "key" : g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request_got = requests.get(base_url, params=params)
    response_json = response.json()
    
    try: hotel_list.append(response_json["results"][0]["name"])
        
    except:
        hotel_list.append("")
hotel_df["Hotel Name"] = hotel_list
hotel_df.dropna()
hotel_df


NameError: name 'response' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer_(markers)
fig

# Display figure
